CHARGEMENT DES DONNÉES

CHARGEMENT DES MODULES

In [ ]:
# IMPORTATIONS DES MODULES
! pip install torch
!pip install gdown
! pip install options
! pip install segmentation-models-pytorch

TELECHARGEMENT DES DATAS + ORGANISATION DES DATAS

In [ ]:
import gdown
import os
import shutil
import zipfile
import os


# Liste des IDs des fichiers
file_ids = [
    '1BqSt4ueO7XAyQ_84mUjswUSJt13ZBuzG',  # val.zip
    '18GuoCuBn48oZKAlEo-LrNwABrFhVALU-',  # train.zip
    '1jj3qJD_grJlgIhUWO09zibRGJe0R4Tn0',  # test.zip
    '1QOjoa-YFvHOgCLXaxvZUlrlidihpmbWU',  # pam_net_F.pth
    '196bR8LdSfVawnSOVmM_YPW25UetucUYX',  # pam_net_A.pth
    '1IVdF5a3e1_7DiSndtMkhpZuCSgDLLFcg'   # best_ckpt.pt
]

# Liste des noms des fichiers
file_names = [
    'val.zip',
    'train.zip',
    'test.zip',
    'pam_net_F.pth',
    'pam_net_A.pth',
    'best_ckpt.pt'
]

# Télécharger chaque fichier en utilisant son ID
for file_id, file_name in zip(file_ids, file_names):
    gdown.download(f'https://drive.google.com/uc?id={file_id}', file_name, quiet=False)

# Créer les dossiers nécessaires
os.makedirs('LEVIR-CD/PAM-PRETRAIN', exist_ok=True)
os.makedirs('LEVIR-CD/BIT_LEVIR', exist_ok=True)
os.makedirs('LEVIR-CD/train', exist_ok=True)
os.makedirs('LEVIR-CD/test', exist_ok=True)
os.makedirs('LEVIR-CD/val', exist_ok=True)

# Déplacer les fichiers dans les sous-dossiers appropriés
shutil.move('pam_net_A.pth', 'LEVIR-CD/PAM-PRETRAIN/pam_net_A.pth')
shutil.move('pam_net_F.pth', 'LEVIR-CD/PAM-PRETRAIN/pam_net_F.pth')
shutil.move('best_ckpt.pt', 'LEVIR-CD/BIT_LEVIR/best_ckpt.pt')
shutil.move('val.zip', 'LEVIR-CD/val.zip')
shutil.move('train.zip', 'LEVIR-CD/train.zip')
shutil.move('test.zip', 'LEVIR-CD/test.zip')






def unzip_file(zip_path, extract_to):
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(extract_to)
    print(f'Décompressé {zip_path} dans {extract_to}')

# Décompression des fichiers
unzip_file('LEVIR-CD/val.zip', 'LEVIR-CD/val')
unzip_file('LEVIR-CD/train.zip', 'LEVIR-CD/train')
unzip_file('LEVIR-CD/test.zip', 'LEVIR-CD/test')
os.remove('LEVIR-CD/val.zip')
os.remove('LEVIR-CD/train.zip')
os.remove('LEVIR-CD/test.zip')
print("Tous les fichiers ont été décompressés et supprimés")





print("Tous les fichiers ont été téléchargés et organisés.")


In [ ]:
import os
import torch
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import torchvision.transforms as transforms

class LEVIRCDClassificationDataset(Dataset):
    def __init__(self, img_dir_before, img_dir_after, mask_dir, transform=None):
        self.img_dir_before = img_dir_before
        self.img_dir_after = img_dir_after
        self.mask_dir = mask_dir
        self.transform = transform
        self.images = os.listdir(img_dir_before)

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        img_name = self.images[idx]
        img_before_path = os.path.join(self.img_dir_before, img_name)
        img_after_path = os.path.join(self.img_dir_after, img_name)
        mask_path = os.path.join(self.mask_dir, img_name)

        img_before = Image.open(img_before_path).convert("RGB")
        img_after = Image.open(img_after_path).convert("RGB")
        mask = Image.open(mask_path).convert("L")  # Grayscale mask

        # Calculer le label binaire : 1 s'il y a un changement, 0 sinon
        label = 1 if torch.sum(transforms.ToTensor()(mask)) > 0 else 0

        if self.transform:
            img_before = self.transform(img_before)
            img_after = self.transform(img_after)

        # Retourner les images avant, après, et le label binaire
        return torch.cat([img_before, img_after], dim=0), label

# Transforms pour normaliser les images
transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
])

# Chemins vers les répertoires de données
img_dir_before = '/content/LEVIR-CD/train/A'
img_dir_after = '/content/LEVIR-CD/train/B'
mask_dir = '/content/LEVIR-CD/train/label'

# Charger les données
train_dataset = LEVIRCDClassificationDataset(img_dir_before, img_dir_after, mask_dir, transform)
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)


ChangeDetectionClassifier(
  (conv1): Conv2d(6, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv3): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=131072, out_features=512, bias=True)
  (fc2): Linear(in_features=512, out_features=1, bias=True)
)


Époque 1/2, Perte: 10.491071428571429
Époque 2/2, Perte: 10.491071428571429
Entraînement terminé.


Test - Accuracy: 0.9219, Precision: 0.9219, Recall: 1.0000, BC: 118.0000, SC: 128.0000, mIoU: 0.4609
Validation - Accuracy: 0.8750, Precision: 0.8750, Recall: 1.0000, BC: 56.0000, SC: 64.0000, mIoU: 0.4375


In [9]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
import segmentation_models_pytorch as smp
from sklearn.metrics import accuracy_score, precision_score, recall_score
import os
from PIL import Image
import torchvision.transforms as transforms
import numpy as np
import matplotlib.pyplot as plt


# Vérifier si un GPU est disponible
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

# Charger le modèle U-Net avec un backbone pré-entraîné (ResNet34)
def get_pretrained_unet():
    model = smp.Unet(
        encoder_name="resnet34",        # Utilise ResNet34 pré-entraîné sur ImageNet
        encoder_weights="imagenet",     # Poids pré-entraînés sur ImageNet
        in_channels=6,                  # Car deux images (avant/après) empilées = 6 canaux
        classes=1,                      # Une sortie binaire (détection de changement)
    )
    return model

# Dataset LEVIR-CD
class LEVIRCDDataset(torch.utils.data.Dataset):
    def __init__(self, imageA_dir, imageB_dir, label_dir, transform=None):
        self.imageA_dir = imageA_dir
        self.imageB_dir = imageB_dir
        self.label_dir = label_dir
        self.transform = transform
        self.image_paths = sorted(os.listdir(imageA_dir))  # Suppose que les fichiers sont ordonnés

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        imageA = Image.open(os.path.join(self.imageA_dir, self.image_paths[idx])).convert('RGB')
        imageB = Image.open(os.path.join(self.imageB_dir, self.image_paths[idx])).convert('RGB')
        label = Image.open(os.path.join(self.label_dir, self.image_paths[idx])).convert('L')

        if self.transform:
            imageA = self.transform(imageA)
            imageB = self.transform(imageB)
            label = self.transform(label)

        # Concaténer les deux images pour obtenir 6 canaux (3 canaux de chaque image)
        combined_image = torch.cat([imageA, imageB], dim=0)
        return combined_image, label

# Configuration des hyperparamètres
num_epochs = 5
learning_rate = 0.001
batch_size = 2

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Resize((1024, 1024)), # Resize the image to match the output size
])

# Charger les données d'entraînement et de validation
train_dataset = LEVIRCDDataset('/content/LEVIR-CD/train/A', '/content/LEVIR-CD/train/B', '/content/LEVIR-CD/train/label', transform=transform)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

val_dataset = LEVIRCDDataset('/content/LEVIR-CD/val/A', '/content/LEVIR-CD/val/B', '/content/LEVIR-CD/val/label', transform=transform)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

# Créer le modèle U-Net
model = get_pretrained_unet().to(device)

# Fonction de perte et optimiseur
criterion = nn.BCEWithLogitsLoss()  # Binary cross-entropy
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Fonction d'entraînement
def train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs):
    for epoch in range(num_epochs):
        model.train()  # Mettre le modèle en mode entraînement
        running_loss = 0.0
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)  # Envoyer les données sur le GPU/CPU
            labels = labels.squeeze(1)
            # Forward pass
            outputs = model(images)
            loss = criterion(outputs, labels.unsqueeze(1).float())

            # Backward pass et optimisation
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

        print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f}")

        # Évaluation sur le jeu de validation après chaque epoch


# Entraîner le modèle
train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs)






Using device: cuda
Epoch [1/5], Loss: 0.1614
Epoch [2/5], Loss: 0.0768
Epoch [3/5], Loss: 0.0726
Epoch [4/5], Loss: 0.0591
Epoch [5/5], Loss: 0.0561


In [10]:


# Fonction de validation
def validate_model_and_save_predictions(model, val_loader, save_dir='predictions'):
    model.eval()
    all_preds = []
    all_labels = []

    # Créer un dossier pour sauvegarder les cartes si nécessaire
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)

    with torch.no_grad():  # Pas de calcul des gradients pendant l'évaluation
        for i, (images, labels) in enumerate(val_loader):
            images = images.to(device)  # Envoyer les images sur le GPU/CPU

            # Faire les prédictions
            outputs = model(images)
            preds = torch.sigmoid(outputs)  # Appliquer la fonction sigmoïde pour obtenir des probabilités
            preds = preds.cpu().numpy() 

            # SAVE SHIFT DETECTION MAP
            for j in range(preds.shape[0]):
                pred = preds[j, 0]  # Extraire la carte de changement pour la jème image du batch
                plt.imsave(os.path.join(save_dir, f'change_map_{i*val_loader.batch_size + j}.png'), pred, cmap='gray')

    


    accuracy = accuracy_score(all_labels, all_preds)
    precision = precision_score(all_labels, all_preds)
    recall = recall_score(all_labels, all_preds)

    print(f"Validation Accuracy: {accuracy:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}")
    print(f"Cartes de détection de changement enregistrées dans le dossier : {save_dir}")


validate_model_and_save_predictions(model, val_loader)

Validation Accuracy: nan, Precision: 0.0000, Recall: 0.0000
Cartes de détection de changement enregistrées dans le dossier : predictions


/usr/local/lib/python3.10/dist-packages/numpy/lib/function_base.py:520: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis, **keepdims_kw)
/usr/local/lib/python3.10/dist-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
